# Metrical Analysis of Sanskrit Ninth Class Verb Forms

## Getting Verbal Roots 

In [1]:
!mkdir -p downloads
!mkdir -p data

In [ ]:
!wget -O downloads/whitney_roots.pdf http://gretil.sub.uni-goettingen.de/gretil_elib/Whi885__Whitney_Roots-ACCENTED.pdf

In [4]:
# install pdftk if not already there. eg: for ubuntu: sudo apt install pdftk
!pdftk downloads/whitney_roots.pdf cat 229 output data/whitney_roots_ninth_class.pdf

In [2]:
# produces data/whitney_roots_ninth_class.txt
!pdftotext data/whitney_roots_ninth_class.pdf

Cleanup the text version manually, fixing formatting and diacritics.

One extra thing we also do is rewrite a form like _mī̆nā_ as _minā/mīnā_, i.e. re-write the variation in the root vowel as two different stem forms explicitly. This helps us visualize and process the variants easily later (note: whitney has only 3 stems here marked like this -- namely _mī̆nā_, _vlī̆nā_ and _dhū̆nī_ -- so we can get away with doing this manually here easily. If there were a lot of these, we could have automated it)

Final results are in [data/whitney_roots_ninth_class_cleaned.txt](data/whitney_roots_ninth_class_cleaned.txt)

In [225]:
# TODO try to get the 9th class forms/roots directly from Lubotksy's concordance?

## Parsing Verbal Roots Info

In [3]:
CLASS_HEADER = "6. nā-class"
EARLIER_LANGUAGE_HEADER = "A. Earlier Language"
EARLIER_AND_LATER_LANGUAGE_HEADER = "B. Earlier and Later Language"
LATER_LANGUAGE_HEADER = "C. Later Language"

NINTH_CLASS_STRONG_MARKER = "ā"
NINTH_CLASS_WEAK_MARKER = "ī"

whitney_roots = []

language_period = None

with open("data/whitney_roots_ninth_class_cleaned.txt", 'r') as whitney_file:
    while line := whitney_file.readline():
        variant_no = None
        attestation_texts = None
        weak_only = False
        
        line = line.rstrip()
        if not line or CLASS_HEADER in line:
            continue    
        elif EARLIER_LANGUAGE_HEADER in line:
            language_period = "Earlier"
            continue
        elif EARLIER_AND_LATER_LANGUAGE_HEADER in line:
            language_period = "Earlier & Later"
            continue
        elif LATER_LANGUAGE_HEADER in line:
            language_period = "Later"
            continue
                
        line_parts = line.split()
        if line_parts[0].isdigit():
            variant_no = line_parts.pop(0)
        stem = line_parts.pop(0)
        if line_parts:
            attestation_texts = " ".join(line_parts)
        
        # this is an assumption safe to make for our data even if there's multiple stem variants
        if stem.endswith(NINTH_CLASS_WEAK_MARKER):
            weak_only = True
        
        # if stem has a variant it will be marked '/'     
        stem_variants = stem.split('/')
        weak_stem_variants = []
        strong_stem_variants = []
        root_variants = []
        
        # populate all the variants above
        for stem_variant in stem_variants:
            # removes the last two chars
            # at this stage, this is a guess only and may not match what is actually used
            # in the grammars/ dictionaries
            root_variants.append(stem_variant[:-2])
            
            if stem_variant.endswith(NINTH_CLASS_WEAK_MARKER):
                weak_stem_variants.append(stem_variant)
                strong_stem_variants.append(stem_variant[:-1] + NINTH_CLASS_STRONG_MARKER)
            else:
                strong_stem_variants.append(stem_variant)
                weak_stem_variants.append(stem_variant[:-1] + NINTH_CLASS_WEAK_MARKER)
        root_variants = sorted(list(set(root_variants)))   
        
        # set the final strings
        root = ' '.join(root_variants)
        strong_stem = ' '.join(strong_stem_variants)
        weak_stem = ' '.join(weak_stem_variants)
        
        # FIXME better place to keep these?
        if root == "pu":
            root = "pū"
        elif root == "ju":
            root = "jū"
        elif root == "ji":
            root = "jī"
        elif root == "vr̥" and variant_no == '2':
            # variant_no 1 'cover' is attested only in AV. 2 is 'choose'
            # FIXME set stem as variant here too and ensure both are searched for
            # but long variant doesn't have any hits here so safe to ignore this here
            #root = "vr̥ ~ vr̥̄"
            # also if we pass it like this, we should also change the stems here
            root = "vr̥ vr̥̄" # other root is vr̥(1, cover which won't match)
        # FIXME better way to pass these exceptions
        # modifying the root here ensures for now we only find matches for the variants
        # used in vedaweb
        # SrI 1 mix: yes
        # SrI 2 boil: not (VB: SrInati..)
        elif root == "śrī" and variant_no == '2': # boil
            root = "śrī 2" # other root is śrī (1, mix) which WILL match
        # aS 1 attain: not (only in mahabhharata 'aSnIs')
        # aS 2 eat yes
        elif root == "aś" and variant_no == '1': # eat
            root = "aśⁱ" # other root is aś (2, attain) which won't match
        # gR as gr̥̄ 1 "sing" attested 
        # gR 2 "swallow" not (only in AVS)
        elif root == "gr̥" and variant_no == '1': # sing
            root = "gr̥̄ 1" # other root is gr̥(2, swallow) which won't match
        #elif variant_no:
            # FIXME assign vedaweb equivalent variant numbers here?
        #    root += ' ' + variant_no 
        
        whitney_roots.append({
            "root_guess": root, # this is really an estimated guess based on the stem 
            "variant_no": variant_no,
            "strong_stem": strong_stem,
            "weak_stem": weak_stem,
            "weak_only": weak_only,
            "attestation_texts": attestation_texts,
            "language_period": language_period,
        })

In [4]:
import pandas

In [5]:
df_whitney_roots = pandas.DataFrame.from_dict(whitney_roots)
df_whitney_roots.to_csv("data/whitney_roots_ninth_class.csv", index=None)
df_whitney_roots.head()

,root_guess,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period
0,i,None,inā,inī,True,V.,Earlier
1,iṣ,None,iṣṇā,iṣṇī,False,None,Earlier
2,ubh,None,ubhnā,ubhnī,False,V.,Earlier
3,uṣ,None,uṣṇā,uṣṇī,False,V.,Earlier
4,kṣi,None,kṣiṇā,kṣiṇī,False,V.B.,Earlier


## Annotating Verbal Roots with Rig Veda Attestations (Manual)

Using Lubotsky's concordance, attestation info is manually added to [data/whitney_roots_ninth_class.csv](data/whitney_roots_ninth_class.csv).

Final results are in [data/roots_ninth_class_manual.csv](data/roots_ninth_class_manual.csv).

In [6]:
#df_roots_manual = pandas.read_csv("data/roots_ninth_class_manual.csv")
df_roots_manual = pandas.read_csv("data/roots_ninth_class_manual.csv", keep_default_na=False)

In [7]:
# TODO remove test df once we have all the annotations
#df_roots_test_manual = df_roots_manual[~df_roots["notes"].isna()]
df_roots_test_manual = df_roots_manual[df_roots_manual["notes"].str.len() > 0]
df_roots_test_manual.head()

,root,variant_no,stem,weak_only,attestation_texts,language_period,rig_veda_weak_attestations,rig_veda_strong_attestations,lubotsky_page_no,notes
1,iṣ,,iṣṇā,False,,Earlier,,1.63.2d,1:,iSnAsi
18,vr̥,1,vr̥ṇī,True,V.,Earlier,1.180.4b 1.67.1b 4.25.3a,,2:1338-1339,avRNItam vRnIte vRnIte(accented - last syll)
37,pu,,punā,False,,Earlier & Later,9.16.3c 9.67.27d,1.133.1a 10.13.3d,1:900-,punIhi puNAmi


## Annotating Verbal Roots with Rig Veda Attestations (Own Search)

### Getting Rig Veda padapatha text (Eichler)

In [ ]:
# http://www.detlef108.de/Rigveda.htm 
# http://www.detlef108.de/Notes-to-the-Rigveda-Page.htm 
!wget -O downloads/rv_padapatha_eichler.html http://www.detlef108.de/RV-Padapatha-TA3-paada-NA-UTF8.html 

In [224]:
# sudo apt install html2text
#!html2text -utf8 -width 3000 -o rv_padapatha.txt rv_padaptaha.html

from bs4 import BeautifulSoup

with open("downloads/rv_padapatha_eichler.html", "r") as input_file:
    soup = BeautifulSoup(input_file)
    
    hymns = []
    
    for para in soup.find_all("p"):
        # ignore the ending notes
        if para.contents[0].name == "span":
            continue
        
        #hymns.append(para.text.rstrip()) # no extra lines between hymns
        hymns.append(para.text)
    
    with open("data/rv_padapatha_eichler.txt", 'w') as f:
        f.write("".join(hymns))

In [222]:
# TODO break the padapatha verse into sub-lines

### Getting Rig Veda padapatha / metrically restored texts

In [ ]:
# https://github.com/cceh/c-salt_vedaweb_sources/tree/master/rigveda/versions
# description of the sources here:
# https://github.com/cceh/c-salt_vedaweb_tei/blob/master/vedaweb_corpus.tei
# https://vedaweb.uni-koeln.de/rigveda/help

!wget -O downloads/rv_padapatha_lubotsky.json https://raw.githubusercontent.com/cceh/c-salt_vedaweb_sources/master/rigveda/versions/lubotsky.json

!wget -O downloads/rv_samhitapatha_vnh.json https://raw.githubusercontent.com/cceh/c-salt_vedaweb_sources/master/rigveda/versions/vnh.json

In [29]:
# make text version from the jsons, with line numbers at the beginning
!python transform_json_corpus.py downloads/rv_padapatha_lubotsky.json

Successfully wrote the sanskrit text to data/rv_padapatha_lubotsky.txt

List of sanskrit chars resolved from the text:

vowels_short: ['a', 'a\\', 'i', 'l̥', 'r̥', 'r̥\\', 'u', 'u\\', '~i', '~u', 'á', 'í', 'ú', 'ŕ̥']
vowels_long: ['ai', 'au', 'au\\', 'aí', 'aú', 'e', 'e\\', 'o', 'o\\', 'r̥̄', 'r̥̄́', 'é', 'ó', 'ā', 'ā\\', 'ā́', 'ī', 'ī\\', 'ī́', 'ū', 'ū\\', 'ū́']
consonants: ['b', 'bh', 'c', 'ch', 'd', 'dh', 'g', 'gh', 'h', 'j', 'jh', 'k', 'kh', 'l', 'm', 'm̐', 'n', 'p', 'ph', 'r', 's', 't', 'th', 'v', 'y', 'ñ', 'ś', 'ḍ', 'ḍh', 'ḥ', 'ḷ', 'ḷh', 'ṁ', 'ṅ', 'ṇ', 'ṣ', 'ṭ', 'ṭh']
special_chars: [' ']
others: []

List of sanskrit chars missing:

vowels_short: ['i\\', 'r̥̀', 'à', 'ì', 'ï', 'ù', 'ü']
vowels_long: ['ai\\', 'aì', 'aù', 'è', 'ò', 'ā̀', 'ī3', 'ī̀', 'ī́3', 'ū3', 'ū̀', 'ū́3']
consonants: []
special_chars: [' ̀', "'"]


In [30]:
# make text version from the jsons, with line numbers at the beginning
!python transform_json_corpus.py downloads/rv_samhitapatha_vnh.json

Successfully wrote the sanskrit text to data/rv_samhitapatha_vnh.txt

List of sanskrit chars resolved from the text:

vowels_short: ['a', 'a\\', 'i', 'i\\', 'l̥', 'r̥', 'u', '~i', '~u', 'á', 'í', 'ú', 'ŕ̥']
vowels_long: ['ai', 'ai\\', 'au', 'aí', 'aú', 'e', 'e\\', 'o', 'o\\', 'r̥̄', 'r̥̄́', 'é', 'ó', 'ā', 'ā\\', 'ā́', 'ī', 'ī3', 'ī́', 'ī́3', 'ū', 'ū́']
consonants: ['b', 'bh', 'c', 'ch', 'd', 'dh', 'g', 'gh', 'h', 'j', 'jh', 'k', 'kh', 'l', 'm', 'm̐', 'n', 'p', 'ph', 'r', 's', 't', 'th', 'v', 'y', 'ñ', 'ś', 'ḍ', 'ḍh', 'ḥ', 'ḷ', 'ḷh', 'ṁ', 'ṅ', 'ṇ', 'ṣ', 'ṭ', 'ṭh']
special_chars: [' ', ' ̀']
others: ["'bh", "'d", "'dh", "'g", "'h", "'j", "'k", "'m", "'n", "'p", "'r", "'s", "'t", "'v", "'y", "'ś"]

List of sanskrit chars missing:

vowels_short: ['r̥\\', 'r̥̀', 'u\\', 'à', 'ì', 'ï', 'ù', 'ü']
vowels_long: ['au\\', 'aì', 'aù', 'è', 'ò', 'ā̀', 'ī\\', 'ī̀', 'ū3', 'ū\\', 'ū̀', 'ū́3']
consonants: []
special_chars: ["'"]


Make sure that the missing chars here are okay to ignore, or that they are just written differently in the text.

### Searching text for ninth-class verbal forms

In [359]:
# TODO search text for ninth-class verbal forms , replicating vedaweb search below?
# use vidyut to identify only finite verbal forms?

## Annotating Verbal Roots with Rig Veda Attestations

In [9]:
# TODO
#!curl -H "Content-Type: application/json" -XPOST https://vedaweb.uni-koeln.de/rigveda/api/search/grammar -d '{}'

import json
import requests
import time

from bs4 import BeautifulSoup
from pprint import pprint

VEDAWEB_API_URL = "https://vedaweb.uni-koeln.de/rigveda/api"

def parse_vedaweb_search_highlight_text(text):
    word_instances = []
    
    for instance_text in BeautifulSoup(text, "lxml").text.split('/'):
        word_gloss = {}
        for prop in instance_text.split(';'):
            prop_parts = prop.split(':')
            prop_name = prop_parts[0].strip()
            prop_value = prop_parts[1].strip()
            
            #if prop_name in ["lemma", "lemma type"]:
            #    continue
                    
            word_gloss[prop_name] = prop_value
            
        word_instances.append(word_gloss)

    return word_instances
        
def search_verb_form_attestations_vedaweb_helper(root, stem=None, results_no=10, results_from=0):
    search_block = {
        "lemma type": "root",
        # make sure we get verbal forms only
        # (i.e. ignore nominal forms like participles which are not marked for person)
        "person": "*", # is present
        "required": True,
        "distance": 0
    }
    
    # TODO make this compulsory
    if stem:
        # handle stems that have spaces to mark variants too!
        # when multiple terms are present separated via space, vedaweb interprets it as an
        # "or" search which is convenient for us
        # TODO break down the search for each variant so that we can track results for each
        # separately? not needed at this stage though
        stem_variants = [ '*' + stem_variant + '*' for stem_variant in stem.split(" ")]
        search_block["term"] = ' '.join(stem_variants)
        #print(search_block["term"])
        
    # for these roots, actually pass in the roots to disambiguate its forms from
    # *mi* and *muṣ*
    # FIXME find a better place for these overrides
    if root in [
        # FIXME auto-figure out by subset (if the stem is a subset already pass root to disambiguate)
        "i", "uṣ",
        "vr̥ vr̥̄", "vr̥", "aśⁱ", "aś", "gr̥̄ 1", "gr̥",
        "śrī", "śrī 2", # FIXME vedaweb does not actually have the second variant at all
    ]:      
        search_block["lemma"] = root
        #print(search_block["lemma"])
    
    response = requests.post(
        VEDAWEB_API_URL + "/search/grammar",
        headers = {"Content-Type": "application/json"},
        json = {
            "mode": "grammar",
            "accents": False,
            "blocks": [search_block],
            "scopes": [],
            "meta": {
                #"hymnAddressee": [],
                #"hymnGroup": [],
                #"strata": [],
                #"stanzaType": [],
                #"lateAdditions": []
            },
            "size": results_no,
            "from": results_from
        }
    )

    # raises an exception on non-200 responses, since we want to know and act on it
    response.raise_for_status()
    
    #pprint(response.request.body)
    #pprint(response.json()["hits"][0])
    
    response_json = response.json()
    
    results = {}
    found_lemmas = set()
    for hit in response_json["hits"]:
        stanza_no = hit["docId"]
        
        words = []
        for word, highlight_text in hit["highlight"].items():
            word_instances = parse_vedaweb_search_highlight_text(highlight_text) 
            for word_gloss in word_instances:
                lemma = word_gloss.pop("lemma")
                #lemma = word_gloss.get("lemma")
                # we don't need this since everything is root for us
                word_gloss.pop("lemma type")
                words.append({
                    "word": word,
                    "gloss": word_gloss
                })
                found_lemmas.add(lemma)
        
        if stanza_no in results:
            # shouldn't happen in our case at all, but just in case
            raise Exception(f"Unexpected, duplicate stanza number found: {stanza_no}")
        else:
            results[stanza_no] = words
    
    # safeguard while testing
    time.sleep(1)
    return {
        "lemmas": list(found_lemmas),
        "count": len(results),
        "total": response_json["total"], 
        "results": results
    }

# TODO remove since we don't use this anymore
ROOT_VARIANT_SUFFIXES = [
    # try with different order, see if we get different results?
    # also need to try all of them?
    # marker of seṭ roots (goes back to PIE roots ending in laryngeals)
    #"ⁱ",
    # numbers used to mark roots of same phonetic shape but different semantics 
    #" 1", " 2", 
    # try this next
    #" 3",
    # up to 3 really should be enough but just in case
    # try with just these for roots where we don't get any hits
    #" 4", " 5",
    # last resort: matches anything that follows
    # needed to catch cases like vr̥ ~ vr̥̄ in vedaweb
    # kind of safe because we also limit our searches by stem where we specify
    # the suffix directly following the root, but to limit false positives,
    # we put this last and specify predicatable variants above
    "*"
]

def search_verb_form_attestations_vedaweb(root, stem=None, results_no=10, results_from=0):
    results = search_verb_form_attestations_vedaweb_helper(root, stem, results_no)
    
#     if results["count"] == 0:
#         for root_variant_suffix in ROOT_VARIANT_SUFFIXES:
#             # our initial root data comes from whitney but how it's represented in vedaweb
#             # may be different so gotta try with other variant forms
#             root_variant = root + root_variant_suffix
#             # useful while debugging
#             #print(
#             #    f"No results found for root:{root} stem:{stem}.",
#             #    f"Retrying with root variant: {root_variant}"
#             #)
#             results = search_verb_form_attestations_vedaweb_helper(root_variant, stem, results_no)

#             # we found a match for a working suffix so stop now
#             if results["count"] > 0:
#                 break
            
            
    # FIXME implement getting all results
    # not needed right now since we get all results at once
    all_results = results["results"]    
    all_results_count = len(all_results)   
        
    if len(results["lemmas"]) > 1:
        raise Exception(
            f"Unexpected, multiple roots found while searching for {root}: {results['lemmas']}"
        )
    
    return {
        "root_vedaweb": results["lemmas"][0] if (all_results_count > 0) else None,
        "count": all_results_count,
        "total": results["total"],
        "results": all_results
    }
        
roots = []

roots_attested_words_by_stanza = {}

for root in whitney_roots:       
    # TODO remove this test filter
    #if root["root_guess"] not in ["iṣ 1", "pū", "vr̥~ vr̥̄"]:
    #
    #if root["root_guess"] not in ["śr̥", "dr̥", "pr̥"]:
    #if root["root_guess"] not in ["i", "uṣ", "mi mī", "muṣ"]:
    #if root["root_guess"] not in ["uṣ", "muṣ"]:
    #if root["root_guess"] not in ["vr̥ vr̥̄", "vr̥", "śrī", "śrī 2", "aśⁱ", "aś", "gr̥̄ 1", "gr̥"]:
    #if root["root_guess"] not in ["pū", "gr̥bh", "iṣ"]:
        #root["strong_attestations"] = ''
        #root["weak_attestations"] = ''
        #roots.append(root)
        #continue                   

    # test cases
    #results = search_verb_form_attestations_vedaweb("iṣ 1", "iṣṇā", 10)
    #results = search_verb_form_attestations_vedaweb("pū", "pun", 10)   
        
    results_strong = search_verb_form_attestations_vedaweb(
        root["root_guess"], root["strong_stem"], 100
    )
    results_weak = search_verb_form_attestations_vedaweb(
        root["root_guess"], root["weak_stem"], 100
    )

    #pprint(results_strong["09.067.27"])
    #pprint(results_strong)
    #pprint(results_weak)
    
    # unlikely to happen but still test for this
    if (results_strong["root_vedaweb"] and results_weak["root_vedaweb"] and
            results_strong["root_vedaweb"] != results_weak["root_vedaweb"]
        ):
        raise Exception(
            f"Root '{results_strong['root_vedaweb']}' inferred from strong-stem results " +
            f"does not match root '{results_weak['root_vedaweb']}' from weak-stem results"
        )
    # both strong and weak results are the same (or one of them is None) by this point
    # so can generalize in either order here  
    results_root_vedaweb = results_weak["root_vedaweb"] or results_strong["root_vedaweb"]
    
    if results_root_vedaweb and root["root_guess"] != results_root_vedaweb:
        # FIXME add notes colummn?: root form originally present as ""
        print(
            f"note: using '{results_root_vedaweb}' for root guess {root['root_guess']}",
            "(as done in vedaweb)"
        )
        # FIXME handle this better without losing the order of the column?
        root["root"] = results_root_vedaweb
        #root["root_variant_vedaweb"] = results_root_vedaweb
    else:
        #root["root_variant_vedaweb"] = ''
        root["root"] = root["root_guess"]
    
    root["strong_attestations"] = " ".join(list(results_strong["results"].keys()))
    root["strong_attestations_total"] = results_strong['total']
    
    root["weak_attestations"] = " ".join(list(results_weak["results"].keys()))
    root["weak_attestations_total"] = results_weak['total']
    
    #root["strong_attestations_data"] = results_strong
    #root["weak_attestations_data"] = results_weak
    
    roots.append(root)
    
    # save full results data, for use later
    roots_attested_words_by_stanza[root["root"]] = {
        "strong": results_strong["results"],
        "weak": results_weak["results"]
    }
    
    # useful diagnostic message
    print(
        # print the variant root form found in vedaweb, if it's there
        #f"{' (' + root['root_variant_vedaweb'] + ')' if root['root_variant_vedaweb'] else ''}", 
        f"{root['root']}: {results_strong['total']} strong, {results_weak['total']} weak attestations"
    )
    
    # so that we don't hammer the api
    #time.sleep(0.5)

#pprint(roots)

temp = set()
dup_roots = [root["root"] for root in roots if root["root"] in temp or temp.add(root["root"])]
if len(dup_roots) > 0:
    # means some other roots also resolved to these during our search
    # we need to ensure our code handles these too now
    raise Exception(f"Final list of roots contain duplicates: {dup_roots}")

i: 0 strong, 0 weak attestations
note: using 'iṣ 1' for root guess iṣ (as done in vedaweb)
iṣ 1: 1 strong, 0 weak attestations
ubh: 2 strong, 0 weak attestations
uṣ: 0 strong, 0 weak attestations
note: using 'kṣī' for root guess kṣi (as done in vedaweb)
kṣī: 3 strong, 0 weak attestations
gr̥: 0 strong, 0 weak attestations
note: using 'gr̥bhⁱ' for root guess gr̥bh (as done in vedaweb)
gr̥bhⁱ: 10 strong, 6 weak attestations
jū: 4 strong, 1 weak attestations
note: using 'jyā' for root guess jī (as done in vedaweb)
jyā: 3 strong, 0 weak attestations
dr̥: 0 strong, 0 weak attestations
drū: 0 strong, 0 weak attestations
note: using 'pr̥̄ 1' for root guess pr̥ (as done in vedaweb)
pr̥̄ 1: 12 strong, 6 weak attestations
pruṣ: 0 strong, 0 weak attestations
bhrī: 0 strong, 0 weak attestations
note: using 'mī 1' for root guess mi mī (as done in vedaweb)
mī 1: 23 strong, 2 weak attestations
note: using 'mr̥̄ 1' for root guess mr̥ (as done in vedaweb)
mr̥̄ 1: 0 strong, 1 weak attestations
ram: 4 st

In [10]:
with open(f"data/roots_ninth_class_attestations.json", 'w') as f:
    json.dump(roots_attested_words_by_stanza, f, indent=2, ensure_ascii=False)

import pandas
df_roots = pandas.DataFrame.from_dict(roots)
df_roots.to_csv("data/roots_ninth_class.csv", index=None)
df_roots.head()

,root_guess,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period,root,strong_attestations,strong_attestations_total,weak_attestations,weak_attestations_total
0,i,None,inā,inī,True,V.,Earlier,i,,0,,0
1,iṣ,None,iṣṇā,iṣṇī,False,None,Earlier,iṣ 1,01.063.02,1,,0
2,ubh,None,ubhnā,ubhnī,False,V.,Earlier,ubh,01.063.04 04.019.04,2,,0
3,uṣ,None,uṣṇā,uṣṇī,False,V.,Earlier,uṣ,,0,,0
4,kṣi,None,kṣiṇā,kṣiṇī,False,V.B.,Earlier,kṣī,04.018.12 10.027.04 10.027.13,3,,0


### Validation: Checking for missing roots

In [11]:
# roots attested in RV
df_roots.query(
    'strong_attestations_total > 0 or weak_attestations_total > 0'
).sort_values("root")

,root_guess,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period,root,strong_attestations,strong_attestations_total,weak_attestations,weak_attestations_total
48,aśⁱ,1,aśnā,aśnī,True,E.,Later,aśⁱ,09.067.31 10.085.03 10.085.04,3,07.073.02,1
39,badh,None,badhnā,badhnī,False,None,Earlier & Later,bandh,10.085.24,1,,0
6,gr̥bh,None,gr̥bhṇā,gr̥bhṇī,False,V.B.,Earlier,gr̥bhⁱ,01.055.02 01.163.02 03.030.05 05.031.07 07.101...,10,09.046.04 09.106.03 10.062.01 10.062.02 10.062...,6
35,gr̥h,None,gr̥hṇā,gr̥hṇī,False,None,Earlier & Later,gr̥hⁱ,04.057.07,1,,0
33,gr̥̄ 1,1,gr̥ṇā,gr̥ṇī,False,None,Earlier & Later,gr̥̄ 1,01.048.04 01.054.07 01.147.02 05.027.03 05.041...,10,01.010.04 01.015.03 01.042.10 01.048.14 01.053...,40
29,hr̥,None,hr̥ṇā,hr̥ṇī,False,V.B.,Earlier,hr̥̄,,0,02.033.15 07.086.03 07.104.14 08.002.19,4
1,iṣ,None,iṣṇā,iṣṇī,False,None,Earlier,iṣ 1,01.063.02,1,,0
8,jī,None,jinā,jinī,False,None,Earlier,jyā,05.034.05 09.055.04 10.027.04,3,,0
36,jā,None,jānā,jānī,False,None,Earlier & Later,jñā,01.163.06 01.164.16 01.164.37 04.004.06 05.061...,8,01.051.08 01.094.08 07.054.01 08.018.15 10.034.04,5
7,jū,None,junā,junī,False,V.,Earlier,jū,01.027.07 01.071.06 01.186.05 07.086.07,4,09.079.02,1


TODO explain cases where root_guess and root differ

eg: iS 'send' being marked with 1 automatically from vedaweb (same as what lubotsky gives)
whitney actually has this as 2 in main root list but since stem does not have variants with it, it's not marked later

In [12]:
# to print without index on the left
#from IPython.display import HTML
#HTML(df_roots.to_html(index=False))

# roots not attested in RV
df_roots.query(
    'strong_attestations_total == 0 and weak_attestations_total == 0'
).sort_values("root")

,root_guess,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period,root,strong_attestations,strong_attestations_total,weak_attestations,weak_attestations_total
31,aś,2,aśnā,aśnī,False,None,Earlier & Later,aś,,0,,0
13,bhrī,None,bhrīṇā,bhrīṇī,False,V.,Earlier,bhrī,,0,,0
51,dhu dhū,None,dhunā dhūnā,dhunī dhūnī,True,C.,Later,dhu dhū,,0,,0
10,drū,None,drūṇā,drūṇī,False,V.B.,Earlier,drū,,0,,0
9,dr̥,None,dr̥ṇā,dr̥ṇī,True,B.,Earlier,dr̥,,0,,0
34,grath,None,grathnā,grathnī,False,B. +,Earlier & Later,grath,,0,,0
5,gr̥,2,gr̥ṇā,gr̥ṇī,False,V.S.,Earlier,gr̥,,0,,0
30,hru,None,hrunā,hrunī,False,V.,Earlier,hru,,0,,0
0,i,None,inā,inī,True,V.,Earlier,i,,0,,0
50,kliś,None,kliśnā,kliśnī,False,None,Later,kliś,,0,,0


TODO Test these with different length of final root vowel? just to see if we catch anything

## Organizing Data by Verse Lines (pādas)

In [13]:
import pandas
#df_roots = pandas.read_csv("data/roots_ninth_class.csv")
df_roots = pandas.read_csv("data/roots_ninth_class.csv", keep_default_na=False)
#df_roots.head()

In [14]:
# TODO remove test df once we have all the annotations
#df_roots_test = df_roots[~df_roots["rig_veda_strong_attestations"].isna()]
#df_roots_test = df_roots[df_roots["rig_veda_strong_attestations"].str.len() > 0]
df_roots_test = df_roots.query('strong_attestations != "" or weak_attestations != ""')
df_roots_test.head()

,root_guess,variant_no,strong_stem,weak_stem,weak_only,attestation_texts,language_period,root,strong_attestations,strong_attestations_total,weak_attestations,weak_attestations_total
1,iṣ,,iṣṇā,iṣṇī,False,,Earlier,iṣ 1,01.063.02,1,,0
2,ubh,,ubhnā,ubhnī,False,V.,Earlier,ubh,01.063.04 04.019.04,2,,0
4,kṣi,,kṣiṇā,kṣiṇī,False,V.B.,Earlier,kṣī,04.018.12 10.027.04 10.027.13,3,,0
6,gr̥bh,,gr̥bhṇā,gr̥bhṇī,False,V.B.,Earlier,gr̥bhⁱ,01.055.02 01.163.02 03.030.05 05.031.07 07.101...,10,09.046.04 09.106.03 10.062.01 10.062.02 10.062...,6
7,jū,,junā,junī,False,V.,Earlier,jū,01.027.07 01.071.06 01.186.05 07.086.07,4,09.079.02,1


In [15]:
from pprint import pprint

In [16]:
rv_lines = []

roots_data = df_roots_test.to_dict("records")

for root in roots_data:
    rv_weak_line_nos = root.pop("weak_attestations").split()
    rv_strong_line_nos = root.pop("strong_attestations").split()
    # FIXME just eliminate these? since these are only stanza
    root.pop("weak_attestations_total")
    root.pop("strong_attestations_total")
    
    #rv_weak_attestations_data = root.pop("rig_veda_weak_attestations_data")
    #rv_strong_attestations_data = root.pop("rig_veda_strong_attestations_data")
    
    weak_stem = root.pop("weak_stem")
    strong_stem = root.pop("strong_stem")
    
    for line_no in rv_weak_line_nos:
        rv_lines.append({"line_no": line_no, "stem": weak_stem, "stem_type": "weak"} | root)
        
    for line_no in rv_strong_line_nos: 
        rv_lines.append({"line_no": line_no, "stem": strong_stem, "stem_type": "strong"} | root)

pprint(rv_lines[0])

{'attestation_texts': '',
 'language_period': 'Earlier',
 'line_no': '01.063.02',
 'root': 'iṣ 1',
 'root_guess': 'iṣ',
 'stem': 'iṣṇā',
 'stem_type': 'strong',
 'variant_no': '',
 'weak_only': False}


### Parsing line numbers

In [17]:
# "1.1.1b" > "01" "001" "02" "b"
# 01.063.02 > "01" "063" "02" ""
def parse_rv_line_no(string):
    line_no_parts = string.split(".")
    
    book = line_no_parts[0].zfill(2)
    hymn = line_no_parts[1].zfill(3)
    
    last_char = line_no_parts[2][-1]
    if last_char.isalpha():
        stanza = line_no_parts[2][:-1] # drop the last char
        pada = last_char
    else:
        stanza = line_no_parts[2]
        pada = ""

    stanza = stanza.zfill(2)

    return {
        "book"    : book,
        "hymn"    : f"{book}.{hymn}",
        "stanza"  : f"{book}.{hymn}.{stanza}",
        "pada"    : f"{book}.{hymn}.{stanza}.{pada}" if pada else ""
        #"pada_id" : pada or ''
    }    

rv_lines = [line | (parse_rv_line_no(line["line_no"])) for line in rv_lines]

pprint(rv_lines[0])

{'attestation_texts': '',
 'book': '01',
 'hymn': '01.063',
 'language_period': 'Earlier',
 'line_no': '01.063.02',
 'pada': '',
 'root': 'iṣ 1',
 'root_guess': 'iṣ',
 'stanza': '01.063.02',
 'stem': 'iṣṇā',
 'stem_type': 'strong',
 'variant_no': '',
 'weak_only': False}


## Annotating Verse Lines

### Downloading annotation data

In [18]:
!mkdir -p downloads/vedaweb

In [19]:
import requests
import time

VEDAWEB_API_URL = "https://vedaweb.uni-koeln.de/rigveda/api"

rv_stanza_nos = sorted(list(set([line["stanza"] for line in rv_lines])))
print(len(rv_stanza_nos))

333


In [21]:
for stanza_no in rv_stanza_nos:
    print(f"Getting data for stanza: {stanza_no}")
    
    # eg: https://vedaweb.uni-koeln.de/rigveda/api/document/id/0100102
    vedaweb_doc_id = stanza_no.replace('.', '')
    vedaweb_doc_url = f"{VEDAWEB_API_URL}/document/id/{vedaweb_doc_id}"
    
    response = requests.get(vedaweb_doc_url)
    # raises an exception on non-200 responses, since we want to know and act on it
    response.raise_for_status()
    
    with open(f"downloads/vedaweb/{stanza_no}.json", 'w') as f:
        f.write(response.text)
    
    # so that we don't hammer the api
    time.sleep(0.5)

print("Done!")

Getting data for stanza: 01.010.04
Getting data for stanza: 01.012.01
Getting data for stanza: 01.013.05
Getting data for stanza: 01.015.02
Getting data for stanza: 01.015.03
Getting data for stanza: 01.023.21
Getting data for stanza: 01.025.01
Getting data for stanza: 01.027.07
Getting data for stanza: 01.032.03
Getting data for stanza: 01.032.04
Getting data for stanza: 01.036.03
Getting data for stanza: 01.039.07
Getting data for stanza: 01.042.05
Getting data for stanza: 01.042.10
Getting data for stanza: 01.044.03
Getting data for stanza: 01.048.04
Getting data for stanza: 01.048.14
Getting data for stanza: 01.051.08
Getting data for stanza: 01.053.02
Getting data for stanza: 01.054.07
Getting data for stanza: 01.055.02
Getting data for stanza: 01.056.06
Getting data for stanza: 01.061.13
Getting data for stanza: 01.063.02
Getting data for stanza: 01.063.04
Getting data for stanza: 01.064.12
Getting data for stanza: 01.067.01
Getting data for stanza: 01.071.06
Getting data for sta

Getting data for stanza: 09.067.26
Getting data for stanza: 09.067.27
Getting data for stanza: 09.067.31
Getting data for stanza: 09.069.03
Getting data for stanza: 09.070.06
Getting data for stanza: 09.071.02
Getting data for stanza: 09.078.01
Getting data for stanza: 09.079.02
Getting data for stanza: 09.083.04
Getting data for stanza: 09.084.01
Getting data for stanza: 09.086.16
Getting data for stanza: 09.097.30
Getting data for stanza: 09.097.41
Getting data for stanza: 09.104.03
Getting data for stanza: 09.106.03
Getting data for stanza: 10.002.04
Getting data for stanza: 10.006.03
Getting data for stanza: 10.009.07
Getting data for stanza: 10.013.03
Getting data for stanza: 10.013.04
Getting data for stanza: 10.015.06
Getting data for stanza: 10.018.13
Getting data for stanza: 10.021.01
Getting data for stanza: 10.023.05
Getting data for stanza: 10.025.03
Getting data for stanza: 10.027.04
Getting data for stanza: 10.027.13
Getting data for stanza: 10.028.02
Getting data for sta

### Enriching the lines with text and metrical info

In [20]:
import json

In [21]:
with open(f"data/roots_ninth_class_attestations.json") as f:
    roots_attested_words_by_stanza = json.load(f)
    
#print(roots_attested_words_by_stanza)

In [22]:
def pada_char_to_no(char):
    # TODO turn this into a dict
    match char:
        case 'a':
            return 0
        case 'b':
            return 1
        case 'c':
            return 2
        case 'd':
            return 3
        case 'e':
            return 4
        case 'f':
            return 5
        case 'g':
            return 6
        case _:
            raise Exception(f"Invalid pada char: {char}")
            
            
def get_stanza_words(stanza_padas):    
    stanza_words = {}
    
    for pada_data in stanza_padas:
        for word_grammar_data in pada_data["grammarData"]:
            word = word_grammar_data["form"]
            
            word_grammar_data_props = word_grammar_data["props"]
            word_position = word_grammar_data_props.pop("position", '')
            word_lemma_type = word_grammar_data_props.pop("lemma type", '')

            word_data = {
                # tracker for when we later search for the actual attested words
                "found": False, 
                "data": {
                    "pada_id": pada_data["id"],
                    # TODO test with this and later eliminate
                    #"pada_index": pada_data["index"], 
                    "pada_label": pada_data["label"],
                    "word": word,
                    "word_position_no": word_grammar_data["index"], # not-zero-indexed!
                    # TODO be careful of this, does not seem to be accurate
                    # (eg: for "punīhi" for 9.67.24 )
                    # TODO use these for checks
                    "word_position": word_position,
                    "word_lemma_type": word_lemma_type,
                    # FIXME pass this and use to validate further
                    #"word_lemma": word_grammar_data["lemma"]
                    "word_props": word_grammar_data_props,
                    # TODO this not needed since all of it is contained in props
                    # but pass and validate they are the same...
                    #"word_gloss": word_tracker_gloss
                }
            }
                
            if word in stanza_words:
                stanza_words[word].append(word_data)
            else:
                # need to use a list since the word may appear multiple times in the stanza
                stanza_words[word] = [word_data]
    
    return stanza_words
    

def get_words_by_pada(stanza_attested_words, stanza_padas, stanza_no=None):
    words_by_pada = []
    
    #pprint(stanza_attested_words)
    #pprint(stanza_padas)
    
    # transform data in stanza_padas to be amenable for searching the attested words
    stanza_words = get_stanza_words(stanza_padas)
    #pprint(stanza_words)
    
    for attested_word_data in stanza_attested_words:
        attested_word = attested_word_data["word"]
        attested_word_gloss = attested_word_data["gloss"]
        
        if attested_word in stanza_words:
            for word_instance in stanza_words[attested_word]:
                # if this word instance was already found, skip to the next one 
                if word_instance["found"]:
                    continue
                    
                word_instance_data = word_instance["data"]
                word_instance_lemma_type = word_instance_data.pop("word_lemma_type")
                
                # FIXME check for lemma too?
                # TODO also ensure this is not causing us to drop valid lines
                if (word_instance_lemma_type and word_instance_lemma_type != "root"):
                    print(
                        f"Skipping an instance of attested word {attested_word} because its lemma type",
                        f"'{word_instance_lemma_type}' is not root"
                    )
                    continue
                    
                if (word_instance_data["word_position"] and
                        "position" in attested_word_gloss and
                        # python "and" operator is short-circuiting so can access "position" below
                        # TODO can we trust this?
                        word_instance_data["word_position"] != attested_word_gloss["position"]
                    ):
                    # even if we skip for a valid instance we will come back to it with a valid position later
                    print(
                        f"Skipping an instance of attested word {attested_word} because it's position '{word_instance_data['word_position']}'",
                        f"does not match the actual attested position '{attested_word_gloss['position']}'"
                    )
                    continue                
                
                # TODO not needed since all of this info is already in word_instance_data
                #word_instance_data["gloss"] = attested_word_gloss
                words_by_pada.append(word_instance_data)
                
                # no need to do this in-place for python!
                word_instance["found"] = True  
                break
        else:
            # TODO handle this better? ok to let go maybe
            raise Exception(
                f"Word {attested_word} was not found in the stanza {stanza_no}: {stanza_padas}"
            )    
            
    # something went wrong and we need to investigate
    if len(words_by_pada) != len(stanza_attested_words):
        raise Exception("No of word instances by pada does not match the input no of word instances attested in the stanza")
    
    return words_by_pada            
            
      
def annotate_line(line):    
    with open(f"downloads/vedaweb/{line['stanza']}.json") as f:
        stanza = json.load(f)
        
        # TODO get pada no for each line (could be multiple) using data in: 
        # roots_attested_words_by_stanza
        # we will be ultimately returning multiple lines here sometimes
        #if line["pada"]:
        #    pada_no = pada_char_to_no(line["pada"][-1])
        #    #pada_no = pada_char_to_no(line["pada_id"])
        #else:
        
        #pada_no = 0
        
        stanza_attested_words = roots_attested_words_by_stanza[line["root"]][line["stem_type"]][line["stanza"]]
            
        words_by_pada = get_words_by_pada(stanza_attested_words, stanza["padas"], line["stanza"])
        #pprint(words_by_pada)
        
        padas = []
        
        for word in words_by_pada:
            pada = line | word
            
            # TODO rename line_no to location everywhere
            pada["line_no"] = pada["stanza"] + "." + pada["pada_id"]
            # this is not needed now
            pada.pop("pada")
            #pada["pada"] = pada["stanza"] + "." + pada["pada_id"]
            
            # FIXME add each of these as a separate field too
            word_props = pada.pop("word_props")
            pada["word_gloss"] = f"{word_props['person']}.{word_props['number']}" + \
                f".{word_props['tense']}.{word_props['mood']}.{word_props['voice']}"
            
            # TODO try out getting index from stanza info directly and see if we still
            # get the same results
            pada_no = pada_char_to_no(pada["pada_id"])
            # TODO testing remove
            #pada_no = 0

            for version in stanza["versions"]:
                if version["id"] == "version_lubotsky":
                    pada["text_padapatha"] = version["form"][pada_no]
                    break
        
            for version in stanza["versions"]:
                if version["id"] == "version_vannootenholland":
                    # TODO deal with * at the begining of the text here?
                    pada["text_samhitapatha"] = version["form"][pada_no]
                    pada["meter_scansion"] = version["metricalData"][pada_no]
                    break
            
            pada["stanza_meter"] = stanza["stanzaType"] or ''
    
            # TODO get these info from hellewig too?
            # historical info
            pada["stanza_strata"] = stanza["strata"]
            pada["stanza_late_addition"] = stanza["lateAdditions"] or ''
            
            # hymn extra metadata (maybe handy)
            pada["hymn_absolute_no"] = stanza["hymnAbs"]
            pada["hymn_addressee"] = stanza["hymnAddressee"]
            pada["hymn_group"] = stanza["hymnGroup"]
        
            # this shouldn't really happen since the results we got were done
            # via stem searches on the padapatha but validate, just in case
            if not any([
                stem_variant in pada["text_padapatha"]
                for stem_variant in pada["stem"].split(" ")
            ]):
                raise Exception(
                    f'Stem {pada["stem"]} not found in the padapatha text "{pada["text_padapatha"]}"'
                )   
        
            padas.append(pada)

        return padas

    
#line_annotated = annotate_line(rv_lines[1])
#pprint(line_annotated)

#rv_lines_annotated = [annotate_line(line) for line in rv_lines]

rv_lines_annotated = []
for line in rv_lines:
    # TODO rename the annotate function here
    rv_lines_annotated.extend(annotate_line(line))

pprint(rv_lines_annotated[0])
print(f"\nTotal number of lines: {len(rv_lines_annotated)}")

{'attestation_texts': '',
 'book': '01',
 'hymn': '01.063',
 'hymn_absolute_no': 63,
 'hymn_addressee': 'Indra',
 'hymn_group': 'Hymns of Nodhas, Descendant of Gotama',
 'language_period': 'Earlier',
 'line_no': '01.063.02.d',
 'meter_scansion': 'SS LLS SSLS LL',
 'pada_id': 'd',
 'pada_label': 'M',
 'root': 'iṣ 1',
 'root_guess': 'iṣ',
 'stanza': '01.063.02',
 'stanza_late_addition': '',
 'stanza_meter': 'Triṣṭubh',
 'stanza_strata': 'A',
 'stem': 'iṣṇā',
 'stem_type': 'strong',
 'text_padapatha': 'púraḥ iṣṇā́si= puruhūta pūrvī́ḥ',
 'text_samhitapatha': 'púra iṣṇā́si puruhūta pūrvī́ḥ',
 'variant_no': '',
 'weak_only': False,
 'word': 'iṣṇā́si',
 'word_gloss': '2.SG.PRS.IND.ACT',
 'word_position': 'intermediate',
 'word_position_no': 2}

Total number of lines: 347


### Saving the Final Line Results

In [23]:
df_rv_lines = pandas.DataFrame.from_dict(rv_lines_annotated)
df_rv_lines.to_csv("data/rv_lines_ninth_class.csv", index=None)
df_rv_lines.head(10)

,line_no,stem,stem_type,root_guess,variant_no,weak_only,attestation_texts,language_period,root,book,...,word_gloss,text_padapatha,text_samhitapatha,meter_scansion,stanza_meter,stanza_strata,stanza_late_addition,hymn_absolute_no,hymn_addressee,hymn_group
0,01.063.02.d,iṣṇā,strong,iṣ,,False,,Earlier,iṣ 1,01,...,2.SG.PRS.IND.ACT,púraḥ iṣṇā́si= puruhūta pūrvī́ḥ,púra iṣṇā́si puruhūta pūrvī́ḥ,SS LLS SSLS LL,Triṣṭubh,A,,63,Indra,"Hymns of Nodhas, Descendant of Gotama"
1,01.063.04.b,ubhnā,strong,ubh,,False,V.,Earlier,ubh,01,...,3.SG.PRS.INJ.ACT,vr̥trám yát vajrin= vr̥ṣakarman ubhnā́ḥ,vr̥tráṁ yád vajrin vr̥ṣakarman ubhnā́ḥ,LL L LL SSLS LL,Triṣṭubh,A,,63,Indra,"Hymns of Nodhas, Descendant of Gotama"
2,04.019.04.c,ubhnā,strong,ubh,,False,V.,Earlier,ubh,04,...,3.SG.IPRF.IND.ACT,dr̥ḷhā́ni aubhnāt= uśámānaḥ ójaḥ,dr̥r̥ḷhā́ni+ aubhnād uśámāna ójo,SLLS LL SSLS LL,,S,,315,Indra,Hymns to Indra
3,04.018.12.d,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,04,...,2.SG.IPRF.IND.ACT,yát prá ákṣiṇāḥ= pitáram pādagŕ̥hya,yát prā́kṣiṇāḥ pitáram pādagŕ̥hya,L LSL SSL LSLS,Triṣṭubh,P,"[Grassmann (G), Arnold (C1)]",314,"Dialogue Between Indra, Aditi and Vamadeva",Hymns to Indra
4,10.027.04.d,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,10,...,1.SG.PRS.INJ.ACT,prá tám kṣiṇām= párvate-_ pādagŕ̥hya,prá táṁ kṣiṇām párvate pādagŕ̥hya,S L SL LSL LSLS,Triṣṭubh,P,[Arnold (C1)],853,Indra,The Vasukra Hymns
5,10.027.13.c,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,kṣī,10,...,3.SG.PRS.IND.ACT,ā́sīnaḥ ūrdhvā́m= upási} kṣiṇāti,ā́sīna ūrdhvā́m upási kṣiṇāti,LLS LL SSL SLS,Triṣṭubh,P,[Arnold (C1)],853,Indra,The Vasukra Hymns
6,09.046.04.b,gr̥bhṇī,weak,gr̥bh,,False,V.B.,Earlier,gr̥bhⁱ,09,...,2.PL.PRS.IND.ACT,śukrā́ gr̥bhṇīta manthínā,śukrā́ gr̥bhṇīta manthínā,LL LLS LSL,Gāyatrī,N,,758,Soma,Tirasci and Other Poets
7,09.106.03.b,gr̥bhṇī,weak,gr̥bh,,False,V.B.,Earlier,gr̥bhⁱ,09,...,3.SG.PRS.INJ.MED,grābhám gr̥bhṇīta sānasím,grābháṁ gr̥bhṇīta sānasím,LL LLS LSL,,A,,818,Soma,The Usnih Group
8,10.062.01.d,gr̥bhṇī,weak,gr̥bh,,False,V.B.,Earlier,gr̥bhⁱ,10,...,2.PL.PRS.IND.ACT,práti gr̥bhṇīta= mānavám} sumedhasaḥ,práti gr̥bhṇīta mānaváṁ sumedhasaḥ,SS LLS LSL SLSL,,C,[Arnold (C1)],888,"All the Gods or the Angiras, Thanksgiving to S...",Nabhanedistha Hymns
9,10.062.02.d,gr̥bhṇī,weak,gr̥bh,,False,V.B.,Earlier,gr̥bhⁱ,10,...,2.PL.PRS.IND.ACT,práti gr̥bhṇīta= mānavám} sumedhasaḥ,práti gr̥bhṇīta mānaváṁ sumedhasaḥ,SS LLS LSL SLSL,,C,[Arnold (C1)],888,"All the Gods or the Angiras, Thanksgiving to S...",Nabhanedistha Hymns


## Validating the data

### Checking for missing roots

In [24]:
import numpy as np

Our starting list of roots:

In [25]:
roots_initial = np.sort(df_roots["root"].unique())
print(f"{len(roots_initial)} roots:\n{roots_initial}")

53 roots:
['aś' 'aśⁱ' 'bandh' 'bhrī' 'dhu dhū' 'drū' 'dr̥' 'grath' 'gr̥' 'gr̥bhⁱ'
 'gr̥hⁱ' 'gr̥̄ 1' 'hru' 'hr̥̄' 'i' 'iṣ 1' 'jyā' 'jñā' 'jū' 'kliś' 'krī'
 'kuṣ' 'kṣī' 'lu' 'mathⁱ' 'mr̥d' 'mr̥̄ 1' 'muṣⁱ' 'mī 1' 'pruṣ' 'prī'
 'pr̥̄ 1' 'puṣ' 'pū' 'ram' 'rī' 'skambhⁱ' 'spr̥' 'stambhⁱ' 'str̥̄' 'subh'
 'sā si' 'ubh' 'uṣ' 'vli vlī' 'vr̥' 'vr̥ vr̥̄' 'śam' 'ścam' 'śrathⁱ' 'śrī'
 'śrī 2' 'śr̥̄ 1']


Attested roots:

In [26]:
roots_present = np.sort(df_rv_lines["root"].unique())
print(f"{len(roots_present)} roots:\n{roots_present}")

30 roots:
['aśⁱ' 'bandh' 'gr̥bhⁱ' 'gr̥hⁱ' 'gr̥̄ 1' 'hr̥̄' 'iṣ 1' 'jyā' 'jñā' 'jū'
 'krī' 'kṣī' 'mathⁱ' 'mr̥̄ 1' 'muṣⁱ' 'mī 1' 'prī' 'pr̥̄ 1' 'pū' 'ram' 'rī'
 'skambhⁱ' 'stambhⁱ' 'str̥̄' 'sā si' 'ubh' 'vr̥ vr̥̄' 'śrathⁱ' 'śrī'
 'śr̥̄ 1']


Missing roots:

In [27]:
roots_absent = np.sort(
    np.setdiff1d(df_roots["root"].unique(), df_rv_lines["root"].unique())
)
print(f"{len(roots_absent)} roots:\n{roots_absent}")

23 roots:
['aś' 'bhrī' 'dhu dhū' 'drū' 'dr̥' 'grath' 'gr̥' 'hru' 'i' 'kliś' 'kuṣ'
 'lu' 'mr̥d' 'pruṣ' 'puṣ' 'spr̥' 'subh' 'uṣ' 'vli vlī' 'vr̥' 'śam' 'ścam'
 'śrī 2']


TODO explain that these roots not being in RV is ok (and expected):
    
* Some roots are attested only as weak stem before vowel (i.e no _nī_ form). e.g.:  [_uṣ_](https://vedaweb.uni-koeln.de/rigveda/view/id/09.097.39)
* Some roots are there in later vedic texts.
* Some roots are already marked as later language in whitney (to be found only in Classical / Epic sanskrit

TODO also try searching by alternate stem/root vowel forms for the missing roots (and also for the attested?), to see if there are actually recorded in those forms

### Checking found word against padapatha text

In [28]:
# from ./meter.py
from meter import clean_lubotsky_padapatha

mismatches = []

no_of_mismatches = 0


for line in rv_lines_annotated:
    padapatha_text_cleaned = clean_lubotsky_padapatha(line["text_padapatha"])
    padapatha_parts = padapatha_text_cleaned.split(' ')
    
    word_from_padapatha = padapatha_parts[line["word_position_no"] - 1]
    
    if line["word"] != word_from_padapatha:
        no_of_mismatches += 1
        print(
            f"{line['line_no']}:{line['word_position_no']}", 
            f"({line['stem']}) {line['word']} ≠ {word_from_padapatha}",
            #f"[{padapatha_text_cleaned}]",
            f"[{line['text_padapatha']}]"
        )
        
print(f"\nFound {no_of_mismatches} mismatches.")

09.104.03.a:1 (punā) punā́ta ≠ punā́tā [punā́tā+ dakṣasā́dhanam]
02.033.13.c:3 (vr̥ṇī) ávr̥ṇīta ≠ ávr̥ṇītā [yā́ni mánuḥ= ávr̥ṇītā+} pitā́ naḥ]
07.033.02.d:3 (vr̥ṇī) avr̥ṇīta ≠ avr̥ṇītā [sutā́t índraḥ= avr̥ṇītā+} vásiṣṭhān]

Found 3 mismatches.


Ignore cases of final vowel lengthening above (especially with imperatives) -- they are not really mismatches.

TODO look at vowel positions for these? but not really in the scope of our investigation